# QC Convenience Functions Demo

This notebook demonstrates the **recommended way** to use the image quality control system through the convenient `manual_qc()` and `auto_qc()` wrapper functions.

## Why Use These Functions?

- **`manual_qc()`**: Automatically sets annotator to your name for human review
- **`auto_qc()`**: Automatically sets annotator to 'automatic' for algorithmic flagging
- **Simpler syntax**: Less typing, fewer parameters to remember
- **Consistent workflow**: Same interface for both manual and automatic QC

In [ ]:
# Setup - Import the convenience functions
import sys
import os
import pandas as pd
from pathlib import Path

# Robust path detection for imports
current_dir = Path.cwd()
print(f"Current directory: {current_dir}")

utils_qc_path = None
if current_dir.name == "segmentation_sandbox":
    utils_qc_path = current_dir / "utils" / "image_quality_qc_utils"
elif current_dir.name == "test":
    utils_qc_path = current_dir.parent
else:
    for parent in current_dir.parents:
        if parent.name == "segmentation_sandbox":
            utils_qc_path = parent / "utils" / "image_quality_qc_utils"
            break

if utils_qc_path and utils_qc_path.exists():
    sys.path.insert(0, str(utils_qc_path))
    print(f"Added to path: {utils_qc_path}")
else:
    print("ERROR: Could not find image_quality_qc_utils directory")

# Import the star functions!
from image_quality_qc_utils import (
    load_qc_data, save_qc_data, get_qc_summary,
    get_flagged_images, get_images_by_flag, get_images_by_annotator,
    QC_FLAGS, manual_qc, auto_qc  # ← These are the convenience functions!
)

print("✓ Successfully imported QC convenience functions!")
print(f"Available QC flags: {list(QC_FLAGS.keys())}")

## Setup Test Data

In [ ]:
# Create test environment
test_data_dir = current_dir / "notebook_convenience_test"
test_data_dir.mkdir(exist_ok=True)

# Initialize with sample images
qc_df = load_qc_data(test_data_dir)

sample_images = [
    {"experiment_id": "20250101", "video_id": "20250101_A01", "image_id": "20250101_A01_t001"},
    {"experiment_id": "20250101", "video_id": "20250101_A01", "image_id": "20250101_A01_t002"},
    {"experiment_id": "20250101", "video_id": "20250101_B01", "image_id": "20250101_B01_t001"},
    {"experiment_id": "20250102", "video_id": "20250102_A01", "image_id": "20250102_A01_t001"},
]

for img in sample_images:
    if len(qc_df) == 0 or img["image_id"] not in qc_df["image_id"].values:
        new_row = pd.DataFrame([{
            "experiment_id": img["experiment_id"],
            "video_id": img["video_id"], 
            "image_id": img["image_id"],
            "qc_flag": None,
            "notes": None,
            "annotator": None
        }])
        qc_df = pd.concat([qc_df, new_row], ignore_index=True)

save_qc_data(qc_df, test_data_dir)
print(f"Setup complete: {len(qc_df)} images ready for QC")
display(qc_df)

## Demo 1: Manual QC with `manual_qc()`

The `manual_qc()` function simplifies human review by automatically setting the annotator field.

In [ ]:
# Example 1: Flag a single image
print("Example 1: Flagging a blurry image with manual_qc()")

qc_df = manual_qc(
    data_dir=test_data_dir,
    annotator="mcolon",  # Your name - set once!
    image_ids=["20250101_A01_t001"],
    qc_flag="BLUR",
    notes="Manual inspection - image appears out of focus",
    overwrite=True
)

print("✓ Flagged image as BLUR")

# Show the result
flagged = qc_df[qc_df["qc_flag"].notna()]
display(flagged[["image_id", "qc_flag", "annotator", "notes"]])

In [ ]:
# Example 2: Flag multiple images at once
print("Example 2: Flagging multiple images with quality issues")

qc_df = manual_qc(
    data_dir=test_data_dir,
    annotator="mcolon",
    image_ids=["20250101_A01_t002", "20250101_B01_t001"],
    qc_flag="DARK",
    notes="Too dark to see embryo features clearly",
    overwrite=True
)

print("✓ Flagged multiple images as DARK")

# Show all manual flags
manual_flags = qc_df[qc_df["annotator"] == "mcolon"]
display(manual_flags[["image_id", "qc_flag", "notes"]])

## Demo 2: Automatic QC with `auto_qc()`

The `auto_qc()` function is perfect for algorithmic quality control - no need to specify annotator!

In [ ]:
# Example 1: Automatic blur detection
print("Example 1: Automatic blur detection with auto_qc()")

qc_df = auto_qc(
    data_dir=test_data_dir,
    image_ids=["20250102_A01_t001"],
    qc_flag="BLUR",
    notes="Automatic: Laplacian variance = 45.2 < threshold (100)",
    overwrite=True
)

print("✓ Automatically flagged image as BLUR")

# Show automatic flags
auto_flags = qc_df[qc_df["annotator"] == "automatic"]
display(auto_flags[["image_id", "qc_flag", "notes"]])

In [ ]:
# Example 2: Automatic PASS flags for good images
print("Example 2: Automatic PASS flags for quality images")

# In real usage, you might process a batch of images that passed quality checks
qc_df = auto_qc(
    data_dir=test_data_dir,
    image_ids=["20250101_A01_t001"],  # Re-evaluate after fixing
    qc_flag="PASS",
    notes="Automatic: All quality metrics within normal range",
    overwrite=True
)

print("✓ Automatically flagged good image as PASS")

# Show all automatic flags
auto_flags = qc_df[qc_df["annotator"] == "automatic"]
display(auto_flags[["image_id", "qc_flag", "notes"]])

## Demo 3: Analyzing Mixed QC Results

The beauty of the system is that manual and automatic QC data coexist in the same file.

In [ ]:
# Get overall summary
summary = get_qc_summary(test_data_dir)
print("Overall QC Summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

# Show breakdown by annotator
print("\nBreakdown by Annotator:")
manual_images = get_images_by_annotator(qc_df, "mcolon")
auto_images = get_images_by_annotator(qc_df, "automatic")
print(f"  Manual (mcolon): {len(manual_images)} images")
print(f"  Automatic: {len(auto_images)} images")

# Show breakdown by flag type
print("\nBreakdown by QC Flag:")
for flag in ['PASS', 'BLUR', 'DARK']:
    flag_images = get_images_by_flag(qc_df, flag)
    if flag_images:
        print(f"  {flag}: {len(flag_images)} images")

## Demo 4: Final Results

See the complete QC data with both manual and automatic annotations.

In [ ]:
# Show final QC data
print("Final QC Data (flagged images only):")
flagged_data = qc_df[qc_df["qc_flag"].notna()]
display(flagged_data[["image_id", "qc_flag", "annotator", "notes"]])

# Save results
output_file = current_dir / "convenience_notebook_results.csv"
qc_df.to_csv(output_file, index=False)
print(f"\nResults saved to: {output_file}")

print("\n🎉 Demo complete!")

## Key Takeaways

### Use `manual_qc()` for Human Review:
```python
manual_qc(data_dir=data_dir, annotator="your_name",
          image_ids=["img1", "img2"], qc_flag="BLUR",
          notes="Manual inspection notes")
```

### Use `auto_qc()` for Algorithmic Flagging:
```python
auto_qc(data_dir=data_dir, 
        image_ids=["img1"], qc_flag="BLUR",
        notes="Automatic: quality_score < threshold")
```

### Benefits:
- **Simpler syntax** - fewer parameters to remember
- **Automatic annotator handling** - no risk of forgetting to set it
- **Consistent workflow** - same interface for manual and automatic QC
- **Mixed data** - manual and automatic flags coexist in same file
- **Easy analysis** - filter by annotator to see human vs algorithmic decisions

### Common Usage Patterns:
1. **Batch manual review**: Flag multiple images with same issue
2. **Automatic pipeline**: Process all images algorithmically 
3. **Video flagging**: Flag entire sequences by video + frames
4. **Mixed workflow**: Combine manual spot-checks with automatic processing